In [8]:
import numpy as np
import os
import warnings
import scipy.sparse as sp
from time import time
from sklearn.metrics import accuracy_score
import tensorflow as tf
from collections import defaultdict
import pickle
import networkx as nx

In [9]:
from easydict import EasyDict

config = {
    'dataset': 'cora',
    'hidden1': 16,
    'epochs': 200,
    'early_stopping': 20,
    'weight_decay': 5e-4,
    'learning_rate': 0.01,
    'dropout': 0.,
    'verbose': False,
    'logging': False,
    'gpu_id': None
}

FLAGS = EasyDict(config)

# 辅助函数

## 数据读取与处理

In [10]:
def load_data_planetoid(dataset):
    """ Load dataset of the splitted version from Planetoid

    Parameters
    ----------
    dataset: string
        name of the dataset

    Returns
    -------
    A_mat,
    X_mat,
    z_vec,
    idx_train,
    idx_val,
    idx_test
    """
    if dataset not in ['citeseer', 'cora', 'pubmed']:
        print("No dataset found!")
    keys = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = defaultdict()
    for key in keys:
        with open('data_split/ind.{}.{}'.format(dataset, key), 'rb') as f:
            objects[key] = pickle.load(f, encoding='latin1')
    test_index = [int(x) for x in open('data_split/ind.{}.test.index'.format(dataset))]
    test_index_sort = np.sort(test_index)
    G = nx.from_dict_of_lists(objects['graph'])

    A_mat = nx.adjacency_matrix(G)
    X_mat = sp.vstack((objects['allx'], objects['tx'])).tolil()
    X_mat[test_index, :] = X_mat[test_index_sort, :]
    z_vec = np.vstack((objects['ally'], objects['ty']))
    z_vec[test_index, :] = z_vec[test_index_sort, :]
    z_vec = z_vec.argmax(1)

    train_idx = range(len(objects['y']))
    val_idx = range(len(objects['y']), len(objects['y']) + 500)
    test_idx = test_index_sort.tolist()

    return A_mat, X_mat, z_vec, train_idx, val_idx, test_idx


def preprocess_graph(adj, c=1):
    """ process the graph
        * options:
        normalization of augmented adjacency matrix
        formulation from convolutional filter
        normalized graph laplacian

    Parameters
    ----------
    adj: a sparse matrix represents the adjacency matrix

    Returns
    -------
    adj_normalized: a sparse matrix represents the normalized laplacian
        matrix
    """
    _adj = adj + c * sp.eye(adj.shape[0])  # Sparse matrix with ones on diagonal 产生对角矩阵
    # _D = sp.diags(_dseq)
    _dseq = _adj.sum(1).A1  # 按行求和后拉直
    _D_half = sp.diags(np.power(_dseq, -0.5)) # 开平方构成对角矩阵
    adj_normalized = _D_half @ _adj @ _D_half # 矩阵乘法
    return adj_normalized.tocsr()    # 转成稀疏矩阵存储

$\hat{D}^{-0.5} \hat{A} \hat{D}^{-0.5}$


In [11]:
# preprocess_graph(A_mat)

In [34]:
# A_mat
t = tf.random.uniform([20])+0.2
tf.cast(tf.floor(t), dtype=tf.bool)

<tf.Tensor: shape=(20,), dtype=bool, numpy=
array([False,  True, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False])>

## 用于处理稀疏矩阵

In [13]:
def sparse_dropout(x, dropout_rate, noise_shape):
    """
    在SparseTensor保留指定非空值。
    x 输入的稀疏矩阵
    noise_shape：x中的元素个数
    
    """
    random_tensor = 1 - dropout_rate
    random_tensor += tf.random.uniform(noise_shape) # 一维数组(noise_shape,)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool) # bool型 一维数组(noise_shape,)
    pre_out = tf.sparse.retain(x, dropout_mask) # 根据dropout_mask的True False 保留x的数据
    return pre_out * (1. / (1 - dropout_rate)) # 数字处理


def sp_matrix_to_sp_tensor(M):
    """
    tf.SparseTensor的作用是构造一个稀疏矩阵类，便于为其他的API提供输入(稀疏矩阵的输入)。
    returns
    X包含 indices,values,dense_shape,dtype
    """
    if not isinstance(M, sp.csr.csr_matrix):
        M = M.tocsr()
    row, col = M.nonzero()  # 非零元素的行和列
    X = tf.SparseTensor(np.mat([row, col]).T, M.data, M.shape) # 函数用于将输入解释为矩阵
    X = tf.cast(X, tf.float32)
    return X

# 定义图卷积层

In [14]:
import tensorflow as tf
from tensorflow.keras import activations, regularizers, constraints, initializers
spdot = tf.sparse.sparse_dense_matmul
dot = tf.matmul


class GCNConv(tf.keras.layers.Layer):

    def __init__(self,
                 units,
                 activation=lambda x: x,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=None,
                 kernel_constraint=None,
                 bias_initializer='zeros',
                 bias_regularizer=None,
                 bias_constraint=None,
                 activity_regularizer=None,
                 **kwargs):
        # 初始化不需要训练的参数
        self.units = units
        # activation=None 使用线性激活函数（等价不使用激活函数）
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        
        # 初始化方法定义了对Keras层设置初始化权重（bias）的方法 glorot_uniform
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        
        # 加载正则化的方法
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        
        # 约束：对权重值施加约束的函数。
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(GCNConv, self).__init__()

    def build(self, input_shape):
        """ GCN has two inputs : [shape(An), shape(X)]
        """
        # gsize = input_shape[0][0]  # graph size
        fdim = input_shape[1][1]  # feature dim
        
        # hasattr 检查该对象self是否有某个属性'weight'
        if not hasattr(self, 'weight'):
            self.weight = self.add_weight(name="weight",
                                          shape=(fdim, self.units),
                                          initializer=self.kernel_initializer,
                                          constraint=self.kernel_constraint,
                                          trainable=True)
        if self.use_bias:
            if not hasattr(self, 'bias'):
                self.bias = self.add_weight(name="bias",
                                            shape=(self.units, ),
                                            initializer=self.bias_initializer,
                                            constraint=self.bias_constraint,
                                            trainable=True)
        super(GCNConv, self).build(input_shape)

    def call(self, inputs):
        """ GCN has two inputs : [An, X]
        """
        self.An = inputs[0]
        self.X = inputs[1]
        # isinstance 函数来判断一个对象是否是一个已知的类型
        if isinstance(self.X, tf.SparseTensor):
            h = spdot(self.X, self.weight)
        else:
            # 二维数组矩阵之间的dot函数运算得到的乘积是矩阵乘积
            h = dot(self.X, self.weight)
        output = spdot(self.An, h)

        if self.use_bias:
            output = tf.nn.bias_add(output, self.bias)

        if self.activation:
            output = self.activation(output)

        return output

# 定义GCN模型

In [15]:
spdot = tf.sparse.sparse_dense_matmul
dot = tf.matmul

tf.get_logger().setLevel('ERROR')


class GCN():
    def __init__(self, An, X, sizes, **kwargs):
        """
        Parameters
        ----------
        An : scipy.sparse matrix
            normalized adjacency matrix
        X : scipy.sparse matrix
            feature matrix
        sizes : list  # [16,7]
            size in each layer
        """
        # 初始化参数
        self.with_relu = True
        self.with_bias = True

        self.lr = FLAGS.learning_rate
        self.dropout = FLAGS.dropout
        self.verbose = FLAGS.verbose
        
        self.An = An
        self.X = X
        self.layer_sizes = sizes
        self.shape = An.shape
        
        # 预处理数据
        self.An_tf = sp_matrix_to_sp_tensor(self.An)
        self.X_tf = sp_matrix_to_sp_tensor(self.X)
        
        # 初始化要用到的层和优化器
        self.layer1 = GCNConv(self.layer_sizes[0], activation='relu')
        self.layer2 = GCNConv(self.layer_sizes[1])
        self.opt = tf.optimizers.Adam(learning_rate=self.lr)

    # 
    def train(self, idx_train, labels_train, idx_val, labels_val):
        """ Train the model
        idx_train : array like
        labels_train : array like
        """
        K = labels_train.max() + 1
        train_losses = []
        val_losses = []
        # use adam to optimize
        for it in range(FLAGS.epochs):
            tic = time()
            with tf.GradientTape() as tape:
                _loss = self.loss_fn(idx_train, np.eye(K)[labels_train])

            # optimize over weights
            grad_list = tape.gradient(_loss, self.var_list)
            grads_and_vars = zip(grad_list, self.var_list)
            self.opt.apply_gradients(grads_and_vars)

            # evaluate on the training
            train_loss, train_acc = self.evaluate(idx_train, labels_train, training=True)
            train_losses.append(train_loss)
            val_loss, val_acc = self.evaluate(idx_val, labels_val, training=False)
            val_losses.append(val_loss)
            toc = time()
            if self.verbose:
                print("iter:{:03d}".format(it),
                      "train_loss:{:.4f}".format(train_loss),
                      "train_acc:{:.4f}".format(train_acc),
                      "val_loss:{:.4f}".format(val_loss),
                      "val_acc:{:.4f}".format(val_acc),
                      "time:{:.4f}".format(toc - tic))
        return train_losses

    def loss_fn(self, idx, labels, training=True):
        """ Calculate the loss function

        Parameters
        ----------
        idx : array like
        labels : array like

        Returns
        -------
        _loss : scalar
        """
        if training:
            # X_tf：X_mat sparse_tensor的矩阵,X.nnz: X中一共多少个元素
            _X = sparse_dropout(self.X_tf, self.dropout, [self.X.nnz])
        else:
            _X = self.X_tf

        self.h1 = self.layer1([self.An_tf, _X])
        if training:
            _h1 = tf.nn.dropout(self.h1, self.dropout)
        else:
            _h1 = self.h1

        self.h2 = self.layer2([self.An_tf, _h1])
        self.var_list = self.layer1.weights + self.layer2.weights
        # calculate the loss base on idx and labels
        # tf.gather 根据索引从参数轴收集切片。
        _logits = tf.gather(self.h2, idx)
        _loss_per_node = tf.nn.softmax_cross_entropy_with_logits(labels=labels,
                                                                 logits=_logits)
        _loss = tf.reduce_mean(_loss_per_node)
        # the weight_dacay only applys to the first layer.
        # weight decay(权值衰减)的作用是调节模型复杂度（体现在 self.layer1.weights）对损失函数的影响，若weight decay很大，则复杂的模型损失函数的值也就大。
        #         Same as the original implementation of GCN.
        # _loss += FLAGS.weight_decay * sum(map(tf.nn.l2_loss, self.var_list))
        # map(fun,series): 根据提供的函数对指定序列做映射。
        # tf.nn.l2_loss: sum(t ** 2) / 2
        _loss += FLAGS.weight_decay * sum(map(tf.nn.l2_loss, self.layer1.weights))
        return _loss

    def evaluate(self, idx, true_labels, training):
        """ Evaluate the model

        Parameters
        ----------
        idx : array like
        true_labels : true labels

        Returns
        -------
        _loss : scalar
        _acc : scalar
        """
        K = true_labels.max() + 1
        _loss = self.loss_fn(idx, np.eye(K)[true_labels], training=training).numpy()
        _pred_logits = tf.gather(self.h2, idx)
        _pred_labels = tf.argmax(_pred_logits, axis=1).numpy()
        _acc = accuracy_score(_pred_labels, true_labels)
        return _loss, _acc

In [16]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) == 0 or FLAGS.gpu_id is None:
    device_id = "/device:CPU:0"
else:
    tf.config.experimental.set_visible_devices(gpus[FLAGS.gpu_id], 'GPU')
    device_id = '/device:GPU:0'

A_mat, X_mat, z_vec, train_idx, val_idx, test_idx = load_data_planetoid(FLAGS.dataset)
A_mat, X_mat, z_vec, train_idx, val_idx, test_idx[:5]

(<2708x2708 sparse matrix of type '<class 'numpy.intc'>'
 	with 10556 stored elements in Compressed Sparse Row format>,
 <2708x1433 sparse matrix of type '<class 'numpy.float32'>'
 	with 49216 stored elements in List of Lists format>,
 array([3, 4, 4, ..., 3, 3, 3], dtype=int64),
 range(0, 140),
 range(140, 640),
 [1708, 1709, 1710, 1711, 1712])

In [17]:
An_mat = preprocess_graph(A_mat)
An_mat

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [29]:
X_mat

<2708x1433 sparse matrix of type '<class 'numpy.float32'>'
	with 49216 stored elements in List of Lists format>

In [42]:
# train_idx
# 小功能1
# K = z_vec.max()+1  # 数字7，用于指定维度
# labels_train = z_vec[train_idx] # 一维数组，用于指定哪个位置为1
# np.eye(K)[labels_train][:2]

# 保留一定的数据
# dropout_rate = 0.2
# noise_shape = [X_mat.nnz]
# x = sp_matrix_to_sp_tensor(X_mat)
# random_tensor = 1 - dropout_rate
# random_tensor += tf.random.uniform(noise_shape) # 一维数组(noise_shape,)
# dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool) # bool型 一维数组(noise_shape,)
# pre_out = tf.sparse.retain(x, dropout_mask) # 根据dropout_mask的True False 保留x的数据



In [ ]:

# N = A_mat.shape[0]
K = z_vec.max() + 1

with tf.device(device_id): # 指定模型运行的具体设备，可以指定运行在GPU还是CUP上，以及哪块GPU上。
    gcn = GCN(An_mat, X_mat, [FLAGS.hidden1, K])  # hidden1=16， out_put: K = 7
    gcn.train(train_idx, z_vec[train_idx], val_idx, z_vec[val_idx])
    test_res = gcn.evaluate(test_idx, z_vec[test_idx], training=False)
    # gcn = GCN(An_mat_diag, X_mat_stack, [FLAGS.hidden1, K])
    # gcn.train(train_idx_recal, z_vec[train_idx], val_idx_recal, z_vec[val_idx])
    # test_res = gcn.evaluate(test_idx_recal, z_vec[test_idx], training=False)
    print("Dataset {}".format(FLAGS.dataset),
          "Test loss {:.4f}".format(test_res[0]),
          "test acc {:.4f}".format(test_res[1]))

In [8]:
if __name__ == "__main__":
    # config the CPU/GPU in TF, assume only one GPU is in use.
    # For multi-gpu setting, please refer to
    #   https://www.tensorflow.org/guide/gpu#using_multiple_gpus

    gpus = tf.config.experimental.list_physical_devices('GPU')
    if len(gpus) == 0 or FLAGS.gpu_id is None:
        device_id = "/device:CPU:0"
    else:
        tf.config.experimental.set_visible_devices(gpus[FLAGS.gpu_id], 'GPU')
        device_id = '/device:GPU:0'

    A_mat, X_mat, z_vec, train_idx, val_idx, test_idx = load_data_planetoid(FLAGS.dataset)
    An_mat = preprocess_graph(A_mat)

    # N = A_mat.shape[0]
    K = z_vec.max() + 1

    with tf.device(device_id):
        gcn = GCN(An_mat, X_mat, [FLAGS.hidden1, K])
        gcn.train(train_idx, z_vec[train_idx], val_idx, z_vec[val_idx])
        test_res = gcn.evaluate(test_idx, z_vec[test_idx], training=False)
        # gcn = GCN(An_mat_diag, X_mat_stack, [FLAGS.hidden1, K])
        # gcn.train(train_idx_recal, z_vec[train_idx], val_idx_recal, z_vec[val_idx])
        # test_res = gcn.evaluate(test_idx_recal, z_vec[test_idx], training=False)
        print("Dataset {}".format(FLAGS.dataset),
              "Test loss {:.4f}".format(test_res[0]),
              "test acc {:.4f}".format(test_res[1]))

Dataset cora Test loss 0.6549 test acc 0.8160
